In [3]:
import pandas as pd

In [5]:
df = pd.read_csv('Cardiac_Surgery_and_Percutaneous_Coronary_Interventions_by_Hospital___Beginning_2008_20250726.csv')
df.head(10)

,Facility ID,Hospital Name,Detailed Region,Region,Procedure,Year of Hospital Discharge,Number of Cases,Number of Deaths,Observed Mortality Rate,Expected Mortality Rate,Risk-Adjusted Mortality Rate,Lower Limit of Confidence Interval,Upper Limit of Confidence Interval,Comparison Results
0,1,Albany Med. Ctr,Capital District,Capital District,All PCI,2016,680,17,2.50,1.52,2.03,1.18,3.26,Rate not different than Statewide Rate
1,1045,White Plains Hospital,NY Metro - New Rochelle,NY Metro - New Rochelle,Non-Emergency PCI,2015,338,1,0.30,0.64,0.34,0.00,1.91,Rate not different than Statewide Rate
2,1438,Bellevue Hospital Ctr,Manhattan,NY Metro - NYC,All PCI,2010,448,4,0.89,0.89,0.84,0.23,2.16,Rate not different than Statewide Rate
3,1439,Beth Israel Med Ctr,Manhattan,NY Metro - NYC,All PCI,2010,1762,11,0.62,0.70,0.75,0.38,1.35,Rate not different than Statewide Rate
4,1178,Bronx-Lebanon-Cncourse,Bronx,NY Metro - NYC,All PCI,2010,65,4,6.15,2.41,2.15,0.58,5.50,Rate not different than Statewide Rate
5,1286,Brookdale Hosp Med Ctr,Kings,NY Metro - NYC,All PCI,2010,338,9,2.66,1.36,1.65,0.75,3.13,Rate not different than Statewide Rate
6,116,Arnot Ogden Med Ctr,Western NY - Rochester,Western NY - Rochester,All PCI,2016,306,3,0.98,1.80,0.68,0.14,1.97,Rate not different than Statewide Rate
7,746,Bassett Medical Center,Capital District,Capital District,All PCI,2016,570,8,1.40,1.09,1.59,0.68,3.13,Rate not different than Statewide Rate
8,1456,Mount Sinai Hospital,Manhattan,NY Metro - NYC,Valve or Valve/CABG,2013-2015,2151,51,2.37,3.09,2.33,1.73,3.06,Rate not different than Statewide Rate
9,1438,Bellevue Hospital Ctr,Manhattan,NY Metro - NYC,All PCI,2016,456,7,1.54,1.70,1.12,0.45,2.31,Rate not different than Statewide Rate


In [ ]:
# Display initial info to understand data types and missing values
print("--- Initial DataFrame Info ---")
df.info()
print("\n--- Initial Head ---")
print(df.head())

# --- 2. Data Cleaning and Type Conversion ---

# Clean column names: remove leading/trailing spaces, replace spaces with underscores
df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('/', '_').str.replace('-', '_')

# Convert relevant columns to numeric types
numeric_cols = [
    'Facility_ID', 'Number_of_Cases', 'Number_of_Deaths',
    'Observed_Mortality_Rate', 'Expected_Mortality_Rate', 'Risk_Adjusted_Mortality_Rate',
    'Lower_Limit_of_Confidence_Interval', 'Upper_Limit_of_Confidence_Interval'
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Check for missing values after initial conversion
print("\n--- Missing Values After Numeric Conversion ---")
print(df.isnull().sum())

def parse_year_range(year_str):
    if pd.isna(year_str):
        return None, None # Handle NaN values
    year_str = str(year_str).strip() # Ensure it's a string and strip whitespace
    if '-' in year_str:
        parts = year_str.split('-')
        try:
            start = int(parts[0])
            end = int(parts[1])
            return start, end
        except ValueError:
            # Handle cases where parts might not be valid integers if data is messy
            return None, None
    else:
        try:
            single_year = int(year_str)
            return single_year, single_year # If it's a single year, start and end are the same
        except ValueError:
            return None, None

# Apply the function to create 'Start_Year' and 'End_Year'
df[['Start_Year', 'End_Year']] = df['Year_of_Hospital_Discharge'].apply(lambda x: pd.Series(parse_year_range(x)))

# For most trending and analysis, using the 'Start_Year' is a practical approach.
df['Mid_Year'] = ((df['Start_Year'] + df['End_Year']) / 2).astype('Int64') # Using Int64 to allow NaN

# --- 4. Feature Engineering: Mortality Flags / Categories ---

# Create a categorical column for 'Comparison_Results'
df['Comparison_Results_Category'] = df['Comparison_Results'].astype('category')

# Create a boolean flag for "Higher than Expected" mortality
df['Is_Higher_Than_Expected_Mortality'] = (df['Comparison_Results'] == 'Rate higher than Statewide Rate')

# Create a boolean flag for "Lower than Expected" mortality
df['Is_Lower_Than_Expected_Mortality'] = (df['Comparison_Results'] == 'Rate lower than Statewide Rate')

# Create a flag for "Not Different from Expected"
df['Is_As_Expected_Mortality'] = (df['Comparison_Results'] == 'Rate not different than Statewide Rate')


# --- 5. Feature Engineering: Rate Differences (useful for analysis) ---

# Difference between Observed and Expected Mortality
df['Observed_vs_Expected_Difference'] = df['Observed_Mortality_Rate'] - df['Expected_Mortality_Rate']

# Difference between Observed and Risk-Adjusted Mortality (if different definitions apply)
df['Observed_vs_RiskAdjusted_Difference'] = df['Observed_Mortality_Rate'] - df['Risk_Adjusted_Mortality_Rate']

# --- 6. Review Final DataFrame ---
print("\n--- Final DataFrame Info After Cleaning and Feature Engineering ---")
df.info()
print("\n--- Final Head with New Columns ---")
print(df.head())
print("\n--- Check Year Range Handling ---")
print(df[['Year_of_Hospital_Discharge', 'Start_Year', 'End_Year', 'Mid_Year']].head(10))

# --- 7. Save Cleaned Data ---
# df.to_csv('cardiac_data_cleaned_engineered.csv', index=False)
# print("\nCleaned and engineered data saved to 'cardiac_data_cleaned_engineered.csv'")

--- Initial DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2812 entries, 0 to 2811
Data columns (total 14 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Facility ID                         2812 non-null   int64  
 1   Hospital Name                       2812 non-null   object 
 2   Detailed Region                     2812 non-null   object 
 3   Region                              2812 non-null   object 
 4   Procedure                           2812 non-null   object 
 5   Year of Hospital Discharge          2812 non-null   object 
 6   Number of Cases                     2812 non-null   int64  
 7   Number of Deaths                    2812 non-null   int64  
 8   Observed Mortality Rate             2812 non-null   float64
 9   Expected Mortality Rate             2812 non-null   float64
 10  Risk-Adjusted Mortality Rate        2812 non-null   float64
 11  Lower Limit 

In [8]:
df.head()

,Facility_ID,Hospital_Name,Detailed_Region,Region,Procedure,Year_of_Hospital_Discharge,Number_of_Cases,Number_of_Deaths,Observed_Mortality_Rate,Expected_Mortality_Rate,...,Comparison_Results,Start_Year,End_Year,Mid_Year,Comparison_Results_Category,Is_Higher_Than_Expected_Mortality,Is_Lower_Than_Expected_Mortality,Is_As_Expected_Mortality,Observed_vs_Expected_Difference,Observed_vs_RiskAdjusted_Difference
0,1,Albany Med. Ctr,Capital District,Capital District,All PCI,2016,680,17,2.50,1.52,...,Rate not different than Statewide Rate,2016,2016,2016,Rate not different than Statewide Rate,False,False,True,0.98,0.47
1,1045,White Plains Hospital,NY Metro - New Rochelle,NY Metro - New Rochelle,Non-Emergency PCI,2015,338,1,0.30,0.64,...,Rate not different than Statewide Rate,2015,2015,2015,Rate not different than Statewide Rate,False,False,True,-0.34,-0.04
2,1438,Bellevue Hospital Ctr,Manhattan,NY Metro - NYC,All PCI,2010,448,4,0.89,0.89,...,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,0.00,0.05
3,1439,Beth Israel Med Ctr,Manhattan,NY Metro - NYC,All PCI,2010,1762,11,0.62,0.70,...,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,-0.08,-0.13
4,1178,Bronx-Lebanon-Cncourse,Bronx,NY Metro - NYC,All PCI,2010,65,4,6.15,2.41,...,Rate not different than Statewide Rate,2010,2010,2010,Rate not different than Statewide Rate,False,False,True,3.74,4.00
